# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Read the "cities.csv" file created in the "WeatherPy" assignment
cities_df = pd.read_csv("output_data/cities.csv")
cities_df


,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,0,Hobart,-42.8794,147.3294,55.26,96,100,17.27,AU,1666721697
1,1,Rikitea,-23.1203,-134.9692,72.91,77,70,20.85,PF,1666721697
2,2,Ushuaia,-54.8000,-68.3000,47.82,49,0,6.91,AR,1666721698
3,3,Ilulissat,69.2167,-51.1000,28.42,93,75,3.44,GL,1666721698
4,4,Airai,-8.9266,125.4092,62.26,93,100,1.34,TL,1666721699
...,...,...,...,...,...,...,...,...,...,...
580,580,Maceió,-9.6658,-35.7353,85.44,65,20,18.41,BR,1666721880
581,581,Port Blair,11.6667,92.7500,80.17,82,37,5.79,IN,1666722106
582,582,Mel'nikovo,56.5558,84.0850,29.25,65,7,10.20,RU,1666722094
583,583,Royan,45.6285,-1.0281,66.78,63,100,6.51,FR,1666722107


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

585

In [33]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations (will be used for heatmap layer)
locations = cities_df[["Lat", "Lng"]]

# # Fill NaN values and convert to float (will be used for heatmap layer)
humidity = cities_df["Humidity (%)"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Set the max intensity to max humidity in our data to caps off the maximum peak intensity
max_humidity = humidity.max()

# Creat the heatmap layer - pass in the required parameters (locations, weights, max intensity ) 
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, max_intensity = max_humidity)

# Add the layer to the map
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create a datafram of cities with ideal weather conditions - use "loc" fuction to filter out the desired weather parameters from the "cities_df"
ideal_weather_df = cities_df.loc[(cities_df["Max Temp (F)"]<80) & (cities_df["Max Temp (F)"]>70) & (cities_df["Wind Speed (mph)"]<10) & (cities_df["Cloudiness (%)"]==0)]

# Clean the dataframe - use "Dropna" function and set how = "any" to drop any rows with null values
ideal_weather_df.dropna(how="any")

# ideal_weather_df.reset_index(inplace=True)
ideal_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
24,24,Grand Gaube,-20.0064,57.6608,73.38,68,0,5.99,MU,1666721719
63,63,Wilmington,34.2257,-77.9447,76.95,72,0,3.44,US,1666721507
104,104,San Clemente,33.4270,-117.6120,77.79,47,0,1.99,US,1666721711
141,141,Nizwá,22.9333,57.5333,79.68,26,0,4.68,OM,1666721801
145,145,Saint-Pierre,-21.3393,55.4781,72.91,73,0,5.75,RE,1666721803
146,146,Salalah,17.0151,54.0924,78.89,57,0,4.61,OM,1666721804
197,197,Jamestown,42.0970,-79.2353,76.59,43,0,9.22,US,1666721835
214,214,Guerrero Negro,27.9769,-114.0611,74.30,33,0,2.51,MX,1666721844
228,228,Hūn,29.1268,15.9477,74.98,29,0,9.13,LY,1666721852
270,270,Karratha,-20.7377,116.8463,73.51,61,0,9.55,AU,1666721877


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Create a new variable for the desired weather condition hotel dataframe
hotel_df = ideal_weather_df

# Add a columns for "Hotel Name" - use "" to specify initial entry
hotel_df["Hotel Name"] = ""
hotel_df

# Get the base URLfrom the Google maps places/ nearby search API. Also create a variable (num_hotels) for number of hotels searched since we do not want to searc more tha 10 hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
num_hotels = 0

# To get the name of hotel for each city in "hotel_df, use "iterrow" function to iterate over DataFrame rows as (index, Series) pairs
# Exception handling - use "Try & accept" code. "Try" will run the API call when there is no error (hotel found with all the listed data point) in the program Whereas the "except" will execute whenever the program encounters some error (e.g. hotel not found for a particular coordinate) in the preceding try block. As s result the prgramm will not halt because of some key or name error.
for index, row in hotel_df.iterrows():
    
    # Write params dictionary
    params = {
        "location": str(row["Lat"]) + "," + str(row["Lng"]),
        "type": "hotel",
        "radius": 5000,
        "key": g_key
    }
    
    try:
        
        if num_hotels >= 10:
            break
        
        response = requests.get(base_url, params).json()
        results = response["results"]
        
        if len(results) == 0:
            print("No hotel found for this city, skipping this city")
            continue
        
        # Getting the firt hotel found name
        hotel_name = results[0]["name"]
        
        # Add hotel name to Hotel Name column for hotel_df
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        
        num_hotels += 1
        
    except (Error):
        print("Error occurred, skipping this city")
        continue

        

In [34]:
# Clean the dataframe - create a new df that has the hotel names
hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
24,24,Grand Gaube,-20.0064,57.6608,73.38,68,0,5.99,MU,1666721719,Goodlands
63,63,Wilmington,34.2257,-77.9447,76.95,72,0,3.44,US,1666721507,Wilmington
104,104,San Clemente,33.4270,-117.6120,77.79,47,0,1.99,US,1666721711,San Clemente
141,141,Nizwá,22.9333,57.5333,79.68,26,0,4.68,OM,1666721801,Nizwa
145,145,Saint-Pierre,-21.3393,55.4781,72.91,73,0,5.75,RE,1666721803,Saint Pierre
146,146,Salalah,17.0151,54.0924,78.89,57,0,4.61,OM,1666721804,Salalah
197,197,Jamestown,42.0970,-79.2353,76.59,43,0,9.22,US,1666721835,Jamestown
214,214,Guerrero Negro,27.9769,-114.0611,74.30,33,0,2.51,MX,1666721844,Guerrero Negro
228,228,Hūn,29.1268,15.9477,74.98,29,0,9.13,LY,1666721852,Houn
270,270,Karratha,-20.7377,116.8463,73.51,61,0,9.55,AU,1666721877,Karratha


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig_hotel = gmaps.figure()
fig_hotel.add_layer(markers)

# Display figure
fig_hotel


Figure(layout=FigureLayout(height='420px'))